# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117946e+02     1.393265e+00
 * time: 0.7725529670715332
     1     1.042078e+01     8.695707e-01
 * time: 2.169071912765503
     2    -1.161274e+01     9.539157e-01
 * time: 2.8425209522247314
     3    -3.385974e+01     6.554008e-01
 * time: 3.8527910709381104
     4    -4.724720e+01     5.256215e-01
 * time: 4.871366024017334
     5    -5.683348e+01     2.083175e-01
 * time: 5.872503042221069
     6    -5.983056e+01     1.080403e-01
 * time: 6.537010908126831
     7    -6.092273e+01     4.878144e-02
 * time: 7.200922012329102
     8    -6.131784e+01     7.477114e-02
 * time: 7.864383935928345
     9    -6.162571e+01     3.225254e-02
 * time: 8.525209903717041
    10    -6.184426e+01     2.298151e-02
 * time: 9.210160970687866
    11    -6.201296e+01     2.066229e-02
 * time: 9.89137601852417
    12    -6.207248e+01     2.045155e-02
 * time: 10.55139708518982
    13    -6.213285e+01     1.220028e-02
 * time: 11.237174034118652
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671086
    AtomicLocal         -18.8557770
    AtomicNonlocal      14.8522701
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485402 
    Xc                  -19.3336832

    total               -62.261666460167
